# 50 persons
## electrodes : 8, 16, 32, 64

In [12]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [13]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [14]:
no_of_patients=50

In [15]:
train=sorted(glob('files2/*.edf'))
train=train[:no_of_patients]
train

['files2/S001R04.edf',
 'files2/S002R04.edf',
 'files2/S003R04.edf',
 'files2/S004R04.edf',
 'files2/S005R04.edf',
 'files2/S006R04.edf',
 'files2/S007R04.edf',
 'files2/S008R04.edf',
 'files2/S009R04.edf',
 'files2/S010R04.edf',
 'files2/S011R04.edf',
 'files2/S012R04.edf',
 'files2/S013R04.edf',
 'files2/S014R04.edf',
 'files2/S015R04.edf',
 'files2/S016R04.edf',
 'files2/S017R04.edf',
 'files2/S018R04.edf',
 'files2/S019R04.edf',
 'files2/S020R04.edf',
 'files2/S021R04.edf',
 'files2/S022R04.edf',
 'files2/S023R04.edf',
 'files2/S024R04.edf',
 'files2/S025R04.edf',
 'files2/S026R04.edf',
 'files2/S027R04.edf',
 'files2/S028R04.edf',
 'files2/S029R04.edf',
 'files2/S030R04.edf',
 'files2/S031R04.edf',
 'files2/S032R04.edf',
 'files2/S033R04.edf',
 'files2/S034R04.edf',
 'files2/S035R04.edf',
 'files2/S036R04.edf',
 'files2/S037R04.edf',
 'files2/S038R04.edf',
 'files2/S039R04.edf',
 'files2/S040R04.edf',
 'files2/S041R04.edf',
 'files2/S042R04.edf',
 'files2/S043R04.edf',
 'files2/S0

In [16]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [17]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [18]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [19]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [20]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(600000, 74950, 600000, 74950)

In [21]:
print(xtest.iloc[:,-1].values)

[-1.0e-05  1.5e-05  6.0e-06 ... -4.4e-05 -4.1e-05 -5.2e-05]


In [22]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000034,-0.000018,-0.000025,-0.000034,-0.000024,-0.000009,-0.000008,-0.000034,-0.000037,-0.000030,...,-0.000021,-0.000024,-0.000015,-0.000025,-0.000026,-0.000031,-0.000030,-0.000021,-0.000026,-0.000049
599996,-0.000052,-0.000042,-0.000040,-0.000046,-0.000043,-0.000022,-0.000021,-0.000047,-0.000048,-0.000042,...,-0.000005,-0.000017,-0.000006,-0.000012,-0.000011,-0.000013,-0.000026,-0.000020,-0.000012,-0.000044
599997,-0.000049,-0.000026,-0.000033,-0.000034,-0.000021,-0.000008,-0.000006,-0.000027,-0.000040,-0.000032,...,-0.000002,-0.000005,0.000006,-0.000005,-0.000006,-0.000008,-0.000012,-0.000011,-0.000006,-0.000035
599998,-0.000036,-0.000033,-0.000033,-0.000035,-0.000031,-0.000016,-0.000014,-0.000040,-0.000038,-0.000034,...,-0.000023,-0.000018,-0.000011,-0.000019,-0.000027,-0.000029,-0.000023,-0.000027,-0.000024,-0.000048


In [23]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [24]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [25]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_4252/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_4252/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_4252/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000034,-0.000018,-0.000025,-0.000034,-0.000024,-0.000009,-0.000008,-0.000034,-0.000037,-0.000030,...,-0.000021,-0.000024,-0.000015,-0.000025,-0.000026,-0.000031,-0.000030,-0.000021,-0.000026,-0.000049
599996,-0.000052,-0.000042,-0.000040,-0.000046,-0.000043,-0.000022,-0.000021,-0.000047,-0.000048,-0.000042,...,-0.000005,-0.000017,-0.000006,-0.000012,-0.000011,-0.000013,-0.000026,-0.000020,-0.000012,-0.000044
599997,-0.000049,-0.000026,-0.000033,-0.000034,-0.000021,-0.000008,-0.000006,-0.000027,-0.000040,-0.000032,...,-0.000002,-0.000005,0.000006,-0.000005,-0.000006,-0.000008,-0.000012,-0.000011,-0.000006,-0.000035
599998,-0.000036,-0.000033,-0.000033,-0.000035,-0.000031,-0.000016,-0.000014,-0.000040,-0.000038,-0.000034,...,-0.000023,-0.000018,-0.000011,-0.000019,-0.000027,-0.000029,-0.000023,-0.000027,-0.000024,-0.000048


In [26]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [27]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [28]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10


18750/18750 [==============================] - 33s 2ms/step - loss: 2.3773 - val_loss: 2.5539
Epoch 2/10
18750/18750 [==============================] - 32s 2ms/step - loss: 1.8357 - val_loss: 2.4673
Epoch 3/10
18750/18750 [==============================] - 32s 2ms/step - loss: 1.7162 - val_loss: 2.6008
Epoch 4/10
18750/18750 [==============================] - 32s 2ms/step - loss: 1.6626 - val_loss: 2.5154
Epoch 5/10
18750/18750 [==============================] - 32s 2ms/step - loss: 1.6293 - val_loss: 2.5408
Epoch 6/10
18750/18750 [==============================] - 32s 2ms/step - loss: 1.6032 - val_loss: 2.5498
Epoch 7/10
18750/18750 [==============================] - 35s 2ms/step - loss: 1.5831 - val_loss: 2.6097
Epoch 8/10
18750/18750 [==============================] - 33s 2ms/step - loss: 1.5696 - val_loss: 2.5869
Epoch 9/10
18750/18750 [==============================] - 42s 2ms/step - loss: 1.5555 - val_loss: 2.6296
Epoch 10/10
18750/18750 [==============================] - 55s 3ms

In [29]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

  34/2343 [..............................] - ETA: 3s  

2343/2343 [==============================] - 4s 2ms/step
              precision    recall  f1-score   support

           0       0.23      0.24      0.24      1499
           1       0.22      0.37      0.28      1499
           2       0.26      0.44      0.33      1499
           3       0.42      0.48      0.45      1499
           4       0.77      0.69      0.73      1499
           5       0.33      0.25      0.29      1499
           6       0.21      0.56      0.31      1499
           7       0.41      0.26      0.32      1499
           8       0.37      0.34      0.35      1499
           9       0.56      0.61      0.58      1499
          10       0.19      0.13      0.16      1499
          11       0.13      0.36      0.19      1499
          12       0.12      0.04      0.05      1499
          13       0.19      0.11      0.14      1499
          14       0.20      0.09      0.13      1499
          15       0.33      0.37      0.35      1499
          16       0.39 

## 0-16

In [30]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [31]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_4252/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_4252/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_4252/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000034,-0.000018,-0.000025,-0.000034,-0.000024,-0.000009,-0.000008,-0.000034,-0.000037,-0.000030,...,-0.000021,-0.000024,-0.000015,-0.000025,-0.000026,-0.000031,-0.000030,-0.000021,-0.000026,-0.000049
599996,-0.000052,-0.000042,-0.000040,-0.000046,-0.000043,-0.000022,-0.000021,-0.000047,-0.000048,-0.000042,...,-0.000005,-0.000017,-0.000006,-0.000012,-0.000011,-0.000013,-0.000026,-0.000020,-0.000012,-0.000044
599997,-0.000049,-0.000026,-0.000033,-0.000034,-0.000021,-0.000008,-0.000006,-0.000027,-0.000040,-0.000032,...,-0.000002,-0.000005,0.000006,-0.000005,-0.000006,-0.000008,-0.000012,-0.000011,-0.000006,-0.000035
599998,-0.000036,-0.000033,-0.000033,-0.000035,-0.000031,-0.000016,-0.000014,-0.000040,-0.000038,-0.000034,...,-0.000023,-0.000018,-0.000011,-0.000019,-0.000027,-0.000029,-0.000023,-0.000027,-0.000024,-0.000048


In [32]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
18750/18750 [==============================] - 57s 3ms/step - loss: 1.7205 - val_loss: 2.2577
Epoch 2/10
18750/18750 [==============================] - 53s 3ms/step - loss: 1.0469 - val_loss: 2.2195
Epoch 3/10
18750/18750 [==============================] - 55s 3ms/step - loss: 0.8897 - val_loss: 2.2777
Epoch 4/10
18750/18750 [==============================] - 51s 3ms/step - loss: 0.8087 - val_loss: 2.3451
Epoch 5/10
18750/18750 [==============================] - 46s 2ms/step - loss: 0.7597 - val_loss: 2.4800
Epoch 6/10
18750/18750 [==============================] - 43s 2ms/step - loss: 0.7266 - val_loss: 2.5097
Epoch 7/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.6996 - val_loss: 2.4669
Epoch 8/10
18750/18750 [==============================] - 41s 2ms/step - loss: 0.6802 - val_loss: 2.4226
Epoch 9/10
18750/18750 [==============================] - 44s 2ms/step - loss: 0.6610 - val_loss: 2.5010
Epoch 10/10
18750/18750 [==============================

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

2343/2343 [==============================] - 3s 1ms/step
              precision    recall  f1-score   support

           0       0.36      0.50      0.42      1499
           1       0.44      0.49      0.46      1499
           2       0.43      0.48      0.46      1499
           3       0.55      0.48      0.51      1499
           4       0.86      0.89      0.87      1499
           5       0.65      0.34      0.44      1499
           6       0.64      0.95      0.77      1499
           7       0.66      0.28      0.40      1499
           8       0.38      0.36      0.37      1499
           9       0.71      0.70      0.71      1499
          10       0.37      0.22      0.27      1499
          11       0.51      0.68      0.58      1499
          12       0.47      0.39      0.42      1499
          13       0.23      0.42      0.30      1499
          14       0.31      0.26      0.28      1499
          15       0.72      0.27      0.40      1499
          16       0.59 

## 0-32

In [36]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [37]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_4252/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_4252/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_4252/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [38]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [39]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [40]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10


2023-10-04 13:44:54.086747: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76800000 exceeds 10% of free system memory.


18750/18750 [==============================] - 43s 2ms/step - loss: 1.1774 - val_loss: 1.9038
Epoch 2/10
  826/18750 [>.............................] - ETA: 40s - loss: 0.5807

18750/18750 [==============================] - 40s 2ms/step - loss: 0.4657 - val_loss: 1.7318
Epoch 3/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.3313 - val_loss: 1.7933
Epoch 4/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.2760 - val_loss: 1.8028
Epoch 5/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.2445 - val_loss: 1.9427
Epoch 6/10
18750/18750 [==============================] - 44s 2ms/step - loss: 0.2229 - val_loss: 1.9116
Epoch 7/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.2063 - val_loss: 1.9929
Epoch 8/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.1937 - val_loss: 1.8743
Epoch 9/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.1825 - val_loss: 1.8914
Epoch 10/10
18750/18750 [==============================] - 44s 2ms/step - loss: 0.1738 - val_loss: 1.9320


In [41]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

  36/2343 [..............................] - ETA: 3s  

2343/2343 [==============================] - 3s 1ms/step
              precision    recall  f1-score   support

           0       0.70      0.75      0.72      1499
           1       0.79      0.70      0.75      1499
           2       0.71      0.51      0.59      1499
           3       0.56      0.55      0.55      1499
           4       0.94      0.97      0.95      1499
           5       0.83      0.87      0.85      1499
           6       0.95      0.91      0.93      1499
           7       0.90      0.68      0.78      1499
           8       0.52      0.79      0.63      1499
           9       0.69      0.82      0.75      1499
          10       0.65      0.54      0.59      1499
          11       0.96      0.93      0.95      1499
          12       0.73      0.80      0.76      1499
          13       0.47      0.75      0.58      1499
          14       0.61      0.70      0.65      1499
          15       0.74      0.31      0.44      1499
          16       0.71 

## 0-64

In [42]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [43]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [44]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [45]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10


2023-10-04 13:52:03.722677: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 153600000 exceeds 10% of free system memory.


18750/18750 [==============================] - 42s 2ms/step - loss: 0.7872 - val_loss: 1.6301
Epoch 2/10
18750/18750 [==============================] - 45s 2ms/step - loss: 0.2821 - val_loss: 1.5910
Epoch 3/10
18750/18750 [==============================] - 45s 2ms/step - loss: 0.2059 - val_loss: 1.5886
Epoch 4/10
18750/18750 [==============================] - 45s 2ms/step - loss: 0.1695 - val_loss: 1.6502
Epoch 5/10
18750/18750 [==============================] - 45s 2ms/step - loss: 0.1453 - val_loss: 1.7106
Epoch 6/10
18750/18750 [==============================] - 43s 2ms/step - loss: 0.1274 - val_loss: 1.7814
Epoch 7/10
18750/18750 [==============================] - 43s 2ms/step - loss: 0.1152 - val_loss: 1.5659
Epoch 8/10
18750/18750 [==============================] - 42s 2ms/step - loss: 0.1045 - val_loss: 1.5328
Epoch 9/10
18750/18750 [==============================] - 45s 2ms/step - loss: 0.0964 - val_loss: 1.5563
Epoch 10/10
18750/18750 [==============================] - 43s 2ms

In [46]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

  37/2343 [..............................] - ETA: 3s  

2343/2343 [==============================] - 3s 1ms/step
              precision    recall  f1-score   support

           0       0.88      0.87      0.88      1499
           1       0.87      0.95      0.91      1499
           2       0.79      0.77      0.78      1499
           3       0.86      0.90      0.88      1499
           4       0.98      0.99      0.99      1499
           5       0.88      0.93      0.91      1499
           6       0.94      0.96      0.95      1499
           7       0.74      0.76      0.75      1499
           8       0.65      0.85      0.74      1499
           9       0.81      0.91      0.86      1499
          10       0.86      0.39      0.54      1499
          11       0.89      0.97      0.93      1499
          12       0.82      0.87      0.85      1499
          13       0.58      0.85      0.69      1499
          14       0.69      0.84      0.76      1499
          15       0.88      0.61      0.72      1499
          16       0.78 